In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandasql as ps
import subprocess
from tqdm import tqdm
import os
from multiprocessing import Pool, Process, Manager
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, AUC, BinaryAccuracy
from tensorflow.keras.layers import  Dense, GRU, Input,Embedding, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam, SGD 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import SGDClassifier
from sklearn.utils import class_weight
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, roc_auc_score,classification_report
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier, plot_importance
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
import random
from category_encoders import TargetEncoder
import random
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import warnings
warnings.filterwarnings('ignore')
import time
from multiprocessing.pool import ThreadPool
import multiprocessing as mp
import gc
from multiprocessing import Pool

In [ ]:
path_train_csv = "../input/riiid-train-data-multiple-formats/riiid_train.feather"
path_question = "../input/riiid-test-answer-prediction/questions.csv"
path_lecture = "../input/riiid-test-answer-prediction/lectures.csv"
path_stat_user = "../input/riid-zefir/results_u_v2.csv"
path_stat_context = "../input/riid-zefir/results_content_v2.csv"
LIST_BD = ["1/","3/","4/","5/","6/","7/"]
LIST_BD2 = ["11/","12/","13/","14/","15/","16/","17/","18/"]
PREOR_TESD_DF = 0
CONTEXT_STAT3 = 0
PREOR_TESD_USER=0
DISCT_USER_STAT2= {}

In [ ]:
QUESTION = pd.read_csv(path_question)
LECTURES = pd.read_csv(path_lecture)
DATA = pd.read_feather(path_train_csv)
UNIQUE_USER = DATA["user_id"].unique().tolist()
# test_user = [*UNIQUE_USER[:100],
#              *UNIQUE_USER[60000:61000],
#              *UNIQUE_USER[110000:110500],
#              *UNIQUE_USER[360000:360500]
#             ]
# train_user = [
#              *UNIQUE_USER[80000:80110],
#              *UNIQUE_USER[380000:380110]
#             ]

USER_STAT = pd.read_csv(path_stat_user)
# CONTEXT_STAT = pd.read_csv(path_stat_context)
# DATA_TRAIN = DATA[DATA.user_id.isin(test_user)]
# DATA_VAL = DATA[DATA.user_id.isin(train_user)]
DATA_TRAIN = DATA[3000000:4300000]
DATA_VAL = DATA[900000:905000]

In [ ]:
# s = DATA_TRAIN[["user_id","content_id"]]
# d = s.set_index("user_id").to_dict()["content_id"]
# US = pd.DataFrame.from_dict(d, orient='index', columns=['foo'])
# US.index.name="user_id"
# d

In [ ]:
s = pd.read_feather("../input/zefir11/0/115")
mas = eval(s["correct"].values[0])
d  = {}
d[115] = (np.mean(mas), np.sum(mas), len(mas))
US = pd.DataFrame.from_dict(d, orient='index', columns=['mean','sum','count'])
US.index.name="user_id"
US

In [ ]:
# s = pd.read_feather("../input/zefir11/0/115")
# # s["user_id"] = 12
# s2 = pd.read_feather("../input/zefir11/100/26769053")
# ss = s.append(s2, ignore_index=True)
# ss["user_id"] = 12
# # s3 = pd.read_feather("../input/zefir11/100/26791627")
# # s4 = pd.read_feather("../input/zefir11/100/26813870")

# # s3 = s3.append(s4, ignore_index=True)
# # s3["user_id"]=13
# # ss = ss.append(s3, ignore_index=True)
# # ss.reset_index(drop=True, inplace=True)
# ss
# # f = eval("["+ss.groupby("user_id")['correct'].apply(lambda x: "*" + (x + ' ,*').cumsum().str.strip()).loc[1][:-3]+"]")
# # ss["cum"] = ss[["user_id","correct"]].groupby("user_id")['correct'].apply(lambda x: "*" + (x + ' ,*').cumsum().str.strip())
# # ss["cum"] = ss.groupby("user_id")["cum"]
# # ss = ss.drop_duplicates(subset=['user_id'], keep='last')
# # eval("["+ss["cum"].values[0][:-3]+"]")
# # for user in ss["user_id"].unique():
# #     arr = ss[ss.user_id==user]["correct"].values
# #     ret = []
# #     for a in arr:
# #         ret = [*ret, *eval(a)]
    
# # np.mean(ret)
# # ss[["user_id","correct"]].groupby("user_id")['correct'].apply(lambda x: "*" + (x + ' ,*').cumsum().str.strip())
# ss["cum"] = ss[["user_id","correct"]].groupby("user_id")['correct'].apply(lambda x:np.mean(eval(x.str.strip())) ) 



In [ ]:
# DICT_USER_ID_PATH = {user_id:path_zefir+dir_in_zefir for path_zefir in LIST_BD for dir_in_zefir in os.listdir("../input/zefir"+path_zefir) for user_id in os.listdir("../input/zefir"+path_zefir+dir_in_zefir)}
DICT_USER_ID_PATH2 = {user_id:path_zefir+dir_in_zefir for path_zefir in LIST_BD2 for dir_in_zefir in os.listdir("../input/zefir"+path_zefir) for user_id in os.listdir("../input/zefir"+path_zefir+dir_in_zefir)}
KEY = DICT_USER_ID_PATH2.keys()

In [ ]:
TODODEL = DATA[:DATA.shape[0]*79//100]
CONTEXT_STAT = TODODEL.loc[TODODEL.content_type_id==0][["content_id","answered_correctly"]].groupby('content_id').agg(["mean","sum","count","std"])
CONTEXT_STAT.columns = ["answered_correctly_content_mean","answered_correctly_content_sum","answered_correctly_content_count","answered_correctly_content_std"]
# CONTEXT_STAT
MEAN_ANSVER_CONTENT_CORRECT = CONTEXT_STAT["answered_correctly_content_mean"].mean()
MEAN_ANSVER_CONTENT_CORRECT_SUM = CONTEXT_STAT["answered_correctly_content_sum"].mean()
MEAN_ANSVER_CONTENT_CORRECT_COUNT = CONTEXT_STAT["answered_correctly_content_count"].mean()
MEAN_ANSVER_CONTENT_CORRECT_STD = CONTEXT_STAT["answered_correctly_content_std"].mean()
del TODODEL, DATA
gc.collect()

In [ ]:
# USER_STAT.reset_index(inplace=True)
USER_STAT.columns = ['user_id',
                     'answered_correctly_mean',
                     'answered_correctly_sum',
                     'answered_correctly_count',
                     'answered_correctly_std',
                     'content_type_id_sum',
                     'content_type_id_count',
                     'timestamp_max',
                     'timestamp_mean',
                     'prior_question_elapsed_time_max',
                     'prior_question_elapsed_time_mean',
                     'prior_question_elapsed_time_std',
                    ]

USER_STAT = USER_STAT[1:]
data_types_dict = {
    'user_id': 'uint64',
    'answered_correctly_mean': 'float32',
    'answered_correctly_sum': 'uint64',
    'answered_correctly_count': 'uint64',
    'answered_correctly_std': 'float32',
    'content_type_id_sum': 'uint64',
    'content_type_id_count': 'uint64',
    'timestamp_max': 'uint64',
    'timestamp_mean': 'float32',
    'prior_question_elapsed_time_max': 'uint64',
    'prior_question_elapsed_time_mean': 'float32',
    'prior_question_elapsed_time_std': 'float32'
}
USER_STAT = USER_STAT.astype(data_types_dict)
USER_STAT = USER_STAT.groupby('user_id').agg({'answered_correctly_mean': 'mean',
                                              'answered_correctly_sum': 'mean',
                                              'answered_correctly_count': 'mean',
                                              'answered_correctly_std': 'mean',
                                              'content_type_id_sum': 'mean',
                                              'content_type_id_count': 'mean',
                                              'timestamp_max': 'mean',
                                              'timestamp_mean': 'mean',
                                              'prior_question_elapsed_time_max': 'mean',
                                              'prior_question_elapsed_time_mean': 'mean',
                                              'prior_question_elapsed_time_std': 'mean',
                                             })

MEAN_ANSVER_USER_CORRECT = USER_STAT["answered_correctly_mean"].mean()
MEAN_SUM = USER_STAT["answered_correctly_sum"].mean()
MEAN_COUNT = USER_STAT["answered_correctly_count"].mean()
MEAN_STD = USER_STAT["answered_correctly_std"].mean()

MEAN_CONTENT_TYPE_ID = USER_STAT["content_type_id_sum"].mean()
MEAN_CONTENT_TYPE_ID_COUNT = USER_STAT["content_type_id_count"].mean()

MEAN_TIMSTEP_MAX = USER_STAT["timestamp_max"].mean()
MEAN_TIMSTEP_MEAN = USER_STAT["timestamp_mean"].mean()

MEAN_PRIOR_QUESTION_TIME_MAX = USER_STAT["prior_question_elapsed_time_max"].mean()
MEAN_PRIOR_QUESTION_TIME_MEAN = USER_STAT["prior_question_elapsed_time_mean"].mean()
MEAN_PRIOR_QUESTION_TIME_STD = USER_STAT["prior_question_elapsed_time_std"].mean()




In [ ]:
column_question = [str(i)+"q" for i in range(189)]
column_question2 = [str(i)+"q2" for i in range(189)]
column_question3 = [str(i)+"q3" for i in range(189)]
column_question4 = [str(i)+"q4" for i in range(189)]
column_question5 = [str(i)+"q5" for i in range(189)]
column_question6 = [str(i)+"q6" for i in range(189)]

column_lectures = [str(i)+"l" for i in range(189)]
part_q=['1_part_q', '2_part_q', '3_part_q', '4_part_q','5_part_q', '6_part_q', '7_part_q']
col_fit2 = ["row_id",
            "timestamp",
            "user_id",
            "content_id",
            "task_container_id",
            "prior_question_elapsed_time",
            "prior_question_had_explanation",
#             "prior_question_had_explanation2",
#             "part_q",
#             "answered_correctly_content_mean"
#             "part",
            
#             'task_container_id2',
#             'task_container_id3',
            
#             'answer_in_the_part',
#             'answer_in_the_part2',
#             'answer_in_the_part3',
#             'answer_in_the_part4',
#             'count_part1',
#             'count_part2',
#             'count_part3',
#             'count_part4'
            
#             'answered_correctly2'
           ]
new_columns = ["count_them",
#             "count_them2",
#             "count_them3",
#             'count_them4',
#             'count_them5',
#             'answer_in_the_past',
#             'answer_in_the_past2',
#             'answer_in_the_past3',
#             'answer_in_the_past4',
#             'percent1',
#             'percent2',
#             'percent3',
#             'percent4',
#             'percent5',
#             'percent6',
#             'percent7'
              ]
# col_fit2 = [*col_fit2,*new_columns]
col_fit2 = [*col_fit2,*["answered_correctly_mean","answered_correctly_sum","answered_correctly_count"]]

col_fit2 = [*col_fit2,*CONTEXT_STAT.columns.tolist()]

# col_fit2 = [*col_fit2,*column_question]
# col_fit2 = [*col_fit2,*["answered_correctly_content_mean"]]
# col_fit2 = [*col_fit2, *["concept", "intention", "question", "solving", "starter"]]


In [ ]:
def segmentation(dataframe,name_columuns,column,separator,discharge=True):
    tags_seq = dataframe[column].values.tolist()
    pp = np.zeros((len(tags_seq), len(name_columuns)), dtype=np.uint8)
    if discharge:
        for i,tag_not_split in tqdm(enumerate(tags_seq)):
            for split_tag in tag_not_split.split(separator):
                rr= int(split_tag)
#                 print(rr)
                pp[i][rr]=1
    else:
        for i,tag_not_split in tqdm(enumerate(tags_seq)):
            mas = tag_not_split.split(separator)
            pp[i][:len(mas)]=mas
    return pp
QUESTION.columns = ["question_id","bundle_id","correct_answer","part_q","tag_q"]
QUESTION["part_q2_str"] = QUESTION['part_q'].astype("str")
QUESTION['tag_q'] = QUESTION['tag_q'].astype("str")
LECTURES['tag'] = LECTURES['tag'].astype("str")
QUESTION['tag_q'] = QUESTION['tag_q'].replace("nan" ,"188")
QUESTION[column_question]=segmentation(QUESTION,column_question,"tag_q"," ",discharge=True)
LECTURES[column_lectures]=segmentation(LECTURES,column_lectures,"tag"," ",discharge=True)
vectorizer = CountVectorizer()
vectorizer2 = CountVectorizer(strip_accents='ascii')
X = vectorizer.fit_transform(LECTURES['type_of'])
LECTURES[vectorizer.get_feature_names()] = X.toarray()

In [ ]:
def sigmentation2(dataframe,column,split=True,prefix=""):
    dataframe[column] = dataframe[column].astype(str)
    dataframe[column]=dataframe[column]+prefix
    TAGS = dataframe[column].unique()
    TAGS_UNIQUE = []
    for tag in tqdm(TAGS):
        if split:
            TAGS_UNIQUE.extend(tag.split())
        else:
            TAGS_UNIQUE.append(tag)
    
    TAGS_UNIQUE = np.unique(TAGS_UNIQUE)
    dataframe[TAGS_UNIQUE]=0
    def calculate_all(row):
        if split:
            row[row[column].split()]=1
        else:
            
            row[row[column]]=1
        return row
    dataframe = dataframe.apply(calculate_all, axis=1)
    dataframe.drop([column], axis=1, inplace=True)
    return dataframe
QUESTION = sigmentation2(QUESTION,"part_q2_str",False,"_part_q")
# LECTURES = sigmentation(LECTURES,"tag",True,"_lec")
# LECTURES = sigmentation(LECTURES,"type_of",False)
# add_question_coumn = list(QUESTION.columns[4:])
# add_question_coumn2 = ["bundle_id","part"]
# add_lecures_columns =  ['tag']
# add_lecures_columns2 = ["part","concept","intention","solving question","starter"]
# add_lecures_columns3 = ["part2","concept","intention","solving question","starter"]

In [ ]:
QUESTION.head()

In [ ]:
DATA_TRAIN["prior_question_had_explanation"].fillna(False, inplace=True)
encoders = LabelEncoder()
encoders.fit(DATA_TRAIN["prior_question_had_explanation"].unique())

In [ ]:
def get_model_sklearn():
#     return SGDClassifier(loss='log')
#     return NearestNeighbors(n_neighbors=10)
#     return MLPClassifier(random_state=1, max_iter=300)
    return XGBClassifier(
    tree_method="hist",
    learning_rate=0.08,
#     gamma=0.1,
    n_estimators=200,
    max_depth=2,
#     min_child_weight=40,
#     subsample=0.87,
#     colsample_bytree=0.95,
#     reg_alpha=0.04,
#     reg_lambda=0.073,
    objective='binary:logistic',
    nthread=4,
    scale_pos_weight=0.8,
    seed=27)

#     return DecisionTreeClassifier(min_samples_leaf=28, min_samples_split=28, max_depth=150)
#     return RandomForestClassifier(n_estimators=200)
#                                   min_samples_leaf=8, min_samples_split=5, max_depth=60)

In [ ]:
QUESTION["content_id"] = QUESTION['question_id']
QUESTION.drop(["question_id"], axis=1, inplace=True)

In [ ]:
QUESTION

In [ ]:
ind_save_data =0
PREOR_TESD_DF = 0
def constructFitDataFrame(df,question, types="TRAIN"):
    global ind_save_data, PREOR_TESD_DF
    df["prior_question_had_explanation"].fillna(False, inplace=True)
    df['prior_question_had_explanation'] = encoders.transform(df['prior_question_had_explanation'])
    
    keys_middle = DISCT_USER_STAT2.keys() 
    if types=="TEST":
        PREIOR = eval(df.iloc[0]["prior_group_answers_correct"])
        if ind_save_data==0:
            PREOR_TESD_DF = df
            ind_save_data=1
        else:
            PREOR_TESD_DF["answered_correctly"] = PREIOR
            for user_id,answered_correctly in PREOR_TESD_DF.loc[PREOR_TESD_DF.content_type_id==0][["user_id","answered_correctly"]].to_numpy():
                if user_id in keys_middle:
                    DISCT_USER_STAT2[user_id].append(answered_correctly)
                else:
                    DISCT_USER_STAT2[user_id] = [answered_correctly]
            PREOR_TESD_DF=df     
    keys_middle = DISCT_USER_STAT2.keys()                
    df = df.loc[df['content_type_id'] == 0]
#     df["is_test"]=1
    
    df.reset_index(drop=True, inplace=True)
    USER_ID = df["user_id"].unique().tolist()
    
  
   
    USER_ID2 = {USER:"../input/zefir"+DICT_USER_ID_PATH2[str(USER)]+"/"+str(USER) for USER in USER_ID if str(USER) in KEY}
    CONTEXT_STAT2 = 0
    DISCT_USER_STAT = {}
    DISCT_USER_STAT2_middle = DISCT_USER_STAT2.copy()
    
#     if type(CONTEXT_STAT3)!=int:
#                 ADD_INFORMATION = CONTEXT_STAT3[CONTEXT_STAT3.user_id.isin(USER_ID)]
#                 DISCT_USER_STAT2 = ADD_INFORMATION.loc[ADD_INFORMATION.content_type_id==0][["user_id","correct"]].set_index("user_id").to_dict()["correct"]
    
    if len(USER_ID2)>0:
        file_list = list(USER_ID2.values())
        ind = 0
        for file in file_list:
            user_id = file.split("/")
            user_id = int(user_id[len(user_id)-1])
            CONTEXT_STAT2 = pd.read_feather(file)
            if user_id in keys_middle:
                mas = [*[int(i) for i in CONTEXT_STAT2["correct"].values[0][1:-1].split(",")],
                       *DISCT_USER_STAT2[user_id]]
                DISCT_USER_STAT[user_id] = (np.mean(mas), np.sum(mas), len(mas))
                USER_ID.remove(user_id)
            else:
                mas = [int(i) for i in CONTEXT_STAT2["correct"].values[0][1:-1].split(",")]
                DISCT_USER_STAT[user_id] = (np.mean(mas), np.sum(mas), len(mas))          
        for u in USER_ID:
            if u in keys_middle:
                mas = DISCT_USER_STAT2[u]
                DISCT_USER_STAT[u] = (np.mean(mas), np.sum(mas), len(mas))
        
        USER_STAT2 = pd.DataFrame.from_dict(DISCT_USER_STAT, orient='index', columns=["answered_correctly_mean",
                                                                "answered_correctly_sum",
                                                                "answered_correctly_count"])
        USER_STAT2.index.name="user_id"
        df = df.merge(USER_STAT2, on=['user_id'], how="left")
    else: #TODO else
        for u in USER_ID:
            if u in keys_middle:
                mas = DISCT_USER_STAT2[u]
                DISCT_USER_STAT[u] = (np.mean(mas), np.sum(mas), len(mas))
        if len(DISCT_USER_STAT):
            USER_STAT2 = pd.DataFrame.from_dict(DISCT_USER_STAT, orient='index', columns=["answered_correctly_mean",
                                                                "answered_correctly_sum",
                                                                "answered_correctly_count"])
            USER_STAT2.index.name="user_id"
            df = df.merge(USER_STAT2, on=['user_id'], how="left")
            
#         #если есть данные из набора test 
#         if type(CONTEXT_STAT3)!=int:
#                 #то получаю их
#                 ADD_INFORMATION = CONTEXT_STAT3[CONTEXT_STAT3.user_id.isin(USER_ID)]
# #                 CONTEXT_STAT2 = ADD_INFORMATION.loc[ADD_INFORMATION.content_type_id==0]
#                 DISCT_USER_STAT2 = ADD_INFORMATION.loc[ADD_INFORMATION.content_type_id==0][["user_id","correct"]].set_index("user_id").to_dict()["correct"]
    
   
                
                #
#                 CONTEXT_STAT2 = CONTEXT_STAT2.append(ADD_INFORMATION, ignore_index=True)
#         CONTEXT_STAT2["cum"] = CONTEXT_STAT2[["user_id","correct"]].groupby("user_id")['correct'].apply(lambda x: "*" + (x + ' ,*').cumsum().str.strip())
#         CONTEXT_STAT2 = CONTEXT_STAT2.drop_duplicates(subset=['user_id'], keep='last')
    
#         for user in CONTEXT_STAT2["user_id"].unique():
#             arr = CONTEXT_STAT2[CONTEXT_STAT2.user_id==user]["correct"].values
#             ret = []
#             for a in arr:
#                 ret = [*ret, *eval(a)]
#             df.loc[df.user_id==user,["answered_correctly_mean",
#                                      "answered_correctly_sum",
#                                      "answered_correctly_count"]] = [np.mean(ret),
#                                                                      np.sum(ret),
#                                                                      len(ret)]
            
        
        # статистика  по ответам пользователя        
#         USER_STAT2 = CONTEXT_STAT2[['user_id','answered_correctly']].groupby("user_id").agg(["mean","sum","count"])
#         USER_STAT2.columns = ["answered_correctly_mean","answered_correctly_sum","answered_correctly_count"] 
#         df = df.merge(USER_STAT2, on=['user_id'], how="left")
        
        
#     if len(USER_ID2)==0:
#         #если есть данные из набора test 
#         if type(CONTEXT_STAT3)!=int:
#                 #то получаю их
#                 ADD_INFORMATION = CONTEXT_STAT3[CONTEXT_STAT3.user_id.isin(USER_ID)]
#                 CONTEXT_STAT2 = ADD_INFORMATION.loc[ADD_INFORMATION.content_type_id==0]
#                 DISCT_USER_STAT2 = ADD_INFORMATION[["user_id","correct"]].set_index("user_id").to_dict()["correct"]
#                 USE
#                 #получаю статистику пользоватлей по ответам
#                 for user in CONTEXT_STAT2["user_id"].unique():
#                     arr = CONTEXT_STAT2[CONTEXT_STAT2.user_id==user]["correct"].values
#                     ret = []
#                     for a in arr:
#                         ret = [*ret, *eval(a)]
                    
#                     df.loc[df.user_id==user,["answered_correctly_mean",
#                                              "answered_correctly_sum",
#                                             "answered_correctly_count"]] = [np.mean(ret),
#                                                                             np.sum(ret),
#                                                                             len(ret)]
#                 USER_STAT2 = CONTEXT_STAT2[['user_id','answered_correctly']].groupby("user_id").agg(["mean","sum","count"])
#                 USER_STAT2.columns = ["answered_correctly_mean","answered_correctly_sum","answered_correctly_count"]
#                 df = df.merge(USER_STAT2, on=['user_id'], how="left")
   
#     if type(CONTEXT_STAT2)!=int:
#         if CONTEXT_STAT2.shape[0]>0:
#             if types=="TEST":
#                 df = CONTEXT_STAT2.append(df, ignore_index=True)
            
#             df = df.merge(QUESTION, how = 'left', on=['content_id'])
#             display(df)
#             все темы на которые отвечал пользователь
#             all_answer_theme = df.groupby('user_id')[column_question].apply(lambda x: x.cumsum())#
#             all_answer_theme["user_id"] = df["user_id"]
            
            #     #предыдущие вопросы
#             past_answer_theme = df.groupby('user_id')[column_question].apply(lambda x:x.shift(1))
#             past_answer_theme["user_id"] = df["user_id"]
#             past_answer_theme["prior_question_had_explanation"] = df["prior_question_had_explanation"]
                #предыдущие темы на которые отвечал пользователь
#             past_all_answer_theme = all_answer_theme.groupby('user_id')[column_question].apply(lambda x:x.shift(1))
#             past_all_answer_theme["user_id"] = df["user_id"]
            #     #предыдущие вопросы на которые видел ответы
#             past_answer_theme_see_answer = past_answer_theme[column_question].multiply(past_answer_theme['prior_question_had_explanation'], axis="index")
#             past_answer_theme_see_answer["user_id"] = df["user_id"]
#             display(past_answer_theme_see_answer)
                #все предыдущие темы вопросов на которые видел ответы
#             cumsum_past_answer_theme_see_answer = past_answer_theme_see_answer.groupby('user_id')[column_question].apply(lambda x: x.cumsum())
    # все выпросы на которые в настоящее время видет ответы
#             question_theme_see_answer = df[column_question].multiply(df['prior_question_had_explanation'], axis="index")
#             question_theme_see_answer["user_id"] = df["user_id"]
#             question_theme_see_answer = question_theme_see_answer.groupby('user_id')[column_question].apply(lambda x: x.cumsum())
#             d=0
# #             d2=0
#             d3=0
# #             d4=0
#             for i in range(189):
#                 theme = df[str(i)+"q"]
#                 see_answer = df["prior_question_had_explanation"]
#                 column = str(i)+"q"
#                 if i==0:
#                     d=theme*past_answer_theme[column].gt(0)*see_answer
# #                     d2 =theme*past_all_answer_theme[column].gt(0)
#                     d3 =theme*cumsum_past_answer_theme_see_answer[column].gt(0)
# #                     d4 = theme*question_theme_see_answer[column].gt(0)
#                 else:
#                     d+=theme*past_answer_theme[column].gt(0)*see_answer
# #                     d2 +=theme*past_all_answer_theme[column].gt(0)
#                     d3 +=theme*cumsum_past_answer_theme_see_answer[column].gt(0)
#                     d4 += theme*question_theme_see_answer[column].gt(0)
#             df["count_them"] = df[column_question].sum(axis=1) #count theme in the question
#             df["count_them2"] = all_answer_theme[column_question].sum(axis=1) # all the theme in the question
#             df['count_them3'] = all_answer_theme[column_question].gt(0).sum(axis=1) # unique count theme in question
#             df["count_them4"] =  df["count_them2"].shift(1)
#             df["count_them5"] =  df["count_them3"].shift(1)

#             df['answer_in_the_past'] = d #видел ли ответы строго на предыдущие вопросы
#             df['answer_in_the_past2'] = d2 # отвечал ли на темы вопросов в прошлом
#             df['answer_in_the_past3'] = d3 # отвечал ли на темы вопросов в прошлом и видел когда-то на них ответы
#             df['answer_in_the_past4'] = d4 # отвечал на эти вопорсы и видел ответы в момент ответа
#             df['percent1'] = d*100/df["count_them"] # процент знания сторого предыдущих тем вопросов
#             df['percent2'] = d2*100/df["count_them"] # процент опыта ответов на прошлые вопросы вне зависимоти от того видел ли ответы
#             df['percent3'] = d3*100/df["count_them"] # процент знания тем вопосов за весь период обучения
#             df['percent4'] = d4*100/df["count_them"] # порцент знания
#             df['percent5'] = df["count_them3"]*100/df["count_them"]
#             df['percent6'] = df["count_them4"]*100/df["count_them"]
#             df['percent7'] = df["count_them5"]*100/df["count_them"]
#         else:
#             df[new_columns]=0
#     else:
#             df[new_columns]=0
            
#     df["prior_question_had_explanation2"] = df.groupby('user_id')["prior_question_had_explanation"].apply(lambda x: x.cumsum())
#     df['task_container_id2'] = df.groupby('user_id')['task_container_id'].transform(lambda x:x.shift(1))
#     df.loc[df.task_container_id==df.task_container_id2,["task_container_id3"]] = 1      
        



#     #все предыдущие темы вопросов на которые видел ответы
#     cumsum_past_answer_theme_see_answer = past_answer_theme_see_answer.groupby('user_id')[column_question].apply(lambda x: x.cumsum())
#     # все выпросы на которые в настоящее время видет ответы
#     question_theme_see_answer = df[column_question].multiply(df['prior_question_had_explanation'], axis="index")
#     question_theme_see_answer["user_id"] = df["user_id"]
#     question_theme_see_answer = question_theme_see_answer.groupby('user_id')[column_question].apply(lambda x: x.cumsum())
    
#     #все part на которые отвечал пользователь
#     all_answer_part = df.groupby('user_id')[part_q].apply(lambda x: x.cumsum())#
#     all_answer_part["user_id"] = df["user_id"]
#     #предыдущие part
#     past_answer_part = df.groupby('user_id')[part_q].apply(lambda x:x.shift(1))
#     past_answer_part["user_id"] = df["user_id"]
#     past_answer_part["prior_question_had_explanation"] = df["prior_question_had_explanation"]
#     #предыдущие part на которые отвечал пользователь
#     past_all_answer_part = all_answer_part.groupby('user_id')[part_q].apply(lambda x:x.shift(1))
#     past_all_answer_part["user_id"] = df["user_id"]
#     #предыдущие part на которые видел ответы
#     past_answer_theme_see_part = past_answer_part[part_q].multiply(past_answer_part['prior_question_had_explanation'], axis="index")
#     past_answer_theme_see_part["user_id"] = df["user_id"]
#     #все предыдущие темы вопросов на которые видел ответы
#     cumsum_past_answer_part_see_answer = past_answer_theme_see_part.groupby('user_id')[part_q].apply(lambda x: x.cumsum())
#     # все выпросы на которые в настоящее время видет ответы
#     question_theme_see_part = df[part_q].multiply(df['prior_question_had_explanation'], axis="index")
#     question_theme_see_part["user_id"] = df["user_id"]
#     question_theme_see_part = question_theme_see_part.groupby('user_id')[part_q].apply(lambda x: x.cumsum())
    
#     d=0
#     d2=0
#     d3=0
#     d4=0
#     part1=0
#     part2=0
#     part3=0
#     part4=0
#     for i in range(1,8):
#         part = df[str(i)+'_part_q']
#         see_answer = df["prior_question_had_explanation"]
#         column = str(i)+'_part_q'
#         if i==0:
#             part1=part*past_answer_part[column].gt(0)*see_answer
#             part2 =part*past_all_answer_part[column].gt(0)
#             part3 =part*cumsum_past_answer_part_see_answer[column].gt(0)
#             part4 = part*question_theme_see_part[column].gt(0)
#         else:
#             part1 +=part*past_answer_part[column].gt(0)*see_answer
#             part2 +=part*past_all_answer_part[column].gt(0)
#             part3 +=part*cumsum_past_answer_part_see_answer[column].gt(0)
#             part4 += part*question_theme_see_part[column].gt(0)
        
        
#     for i in range(189):
#         theme = df[str(i)+"q"]
#         see_answer = df["prior_question_had_explanation"]
#         column = str(i)+"q"
#         if i==0:
#             d=theme*past_answer_theme[column].gt(0)*see_answer
#             d2 =theme*past_all_answer_theme[column].gt(0)
#             d3 =theme*cumsum_past_answer_theme_see_answer[column].gt(0)
#             d4 = theme*question_theme_see_answer[column].gt(0)
#         else:
#             d+=theme*past_answer_theme[column].gt(0)*see_answer
#             d2 +=theme*past_all_answer_theme[column].gt(0)
#             d3 +=theme*cumsum_past_answer_theme_see_answer[column].gt(0)
#             d4 += theme*question_theme_see_answer[column].gt(0)
#     df["count_them"] = df[column_question].sum(axis=1) #count theme in the question
#     df["count_them2"] = all_answer_theme[column_question].sum(axis=1) # all the theme in the question
#     df['count_them3'] = all_answer_theme[column_question].gt(0).sum(axis=1) # unique count theme in question
#     df["count_them4"] =  df["count_them2"].shift(1)
#     df["count_them5"] =  df["count_them3"].shift(1)
    
#     df['answer_in_the_past'] = d #видел ли ответы строго на предыдущие вопросы
#     df['answer_in_the_past2'] = d2 # отвечал ли на темы вопросов в прошлом
#     df['answer_in_the_past3'] = d3 # отвечал ли на темы вопросов в прошлом и видел когда-то на них ответы
#     df['answer_in_the_past4'] = d4 # отвечал на эти вопорсы и видел ответы в момент ответа
#     df['percent1'] = d*100/df["count_them"] # процент знания сторого предыдущих тем вопросов
#     df['percent2'] = d2*100/df["count_them"] # процент опыта ответов на прошлые вопросы вне зависимоти от того видел ли ответы
#     df['percent3'] = d3*100/df["count_them"] # процент знания тем вопосов за весь период обучения
#     df['percent4'] = d4*100/df["count_them"] # порцент знания
#     df['percent5'] = df["count_them3"]*100/df["count_them"]
#     df['percent6'] = df["count_them4"]*100/df["count_them"]
#     df['percent7'] = df["count_them5"]*100/df["count_them"]
    
    
#     df['answer_in_the_part'] = part1 #видел ли ответы строго на предыдущие part
#     df['answer_in_the_part2'] = part2 #видел ли ответы строго на предыдущие part
#     df['answer_in_the_part3'] = part3 #видел ли ответы строго на предыдущие part
#     df['answer_in_the_part4'] = part4 #видел ли ответы строго на предыдущие part
    
# #     df["count_part"] = df[column_question].sum(axis=1) #count theme in the question
#     df["count_part1"] = all_answer_part[part_q].sum(axis=1) # all the theme in the question
#     df['count_part2'] = all_answer_part[part_q].gt(0).sum(axis=1) # unique count theme in question
#     df["count_part3"] =  df["count_part1"].shift(1)
#     df["count_part4"] =  df["count_part2"].shift(1)
    
    
    
#     

    
   
#     df = df[df.is_test==1]        
    df = df.merge(CONTEXT_STAT, on=['content_id'],  how="left")
    RAND_CORR = random.uniform(MEAN_ANSVER_USER_CORRECT, 0.65)
    df["answered_correctly_mean"].fillna(RAND_CORR,inplace=True)
    df["answered_correctly_sum"].fillna(MEAN_SUM,inplace=True)
    df["answered_correctly_count"].fillna(MEAN_COUNT,inplace=True)
    df["answered_correctly_content_mean"].fillna(MEAN_ANSVER_CONTENT_CORRECT,inplace=True)
    df["answered_correctly_content_sum"].fillna(MEAN_ANSVER_CONTENT_CORRECT_SUM,inplace=True)
    df["answered_correctly_content_count"].fillna(MEAN_ANSVER_CONTENT_CORRECT_COUNT,inplace=True)
    df["answered_correctly_content_std"].fillna(MEAN_ANSVER_CONTENT_CORRECT_STD,inplace=True)
    df.fillna(0, inplace=True)
    return df

In [ ]:
%time
train = constructFitDataFrame(DATA_TRAIN,QUESTION)
train[col_fit2]

In [ ]:
%time
CONTEXT_STAT3 = 0
print(DATA_VAL[DATA_VAL.content_type_id==0].shape)
val = constructFitDataFrame(DATA_VAL,QUESTION)
train_x = train[col_fit2]
val_x = val[col_fit2]
print(train_x.info())
print(val_x.info())
train_y = train["answered_correctly"]
val_y = val["answered_correctly"]
print(val.shape)

In [ ]:
# # # d = train[column_question]*train[column_question2]
# train[train.user_id==115][["tag_q","answer_in_the_past",'count_them2',"count_them4","answer_in_the_past2",'answer_in_the_past3',"prior_question_had_explanation"]]

In [ ]:
model = get_model_sklearn()
model.fit(train_x,train_y)
print("ROC AUC TRAIN: ",make_scorer(roc_auc_score, needs_proba=True)(model, train_x, train_y))
pred = model.predict_proba(val_x)
print("ROC AUC: ",roc_auc_score(val_y, pred[:,1]))
print(pred.shape)

In [ ]:
# CONTEXT_STAT3 = 0
# sh = DATA_VAL.shape[0]
# preds = []
# for step in tqdm(range(0,sh,19)):
#     pred_data = DATA_VAL[step:step+19]
#     pred_data = constructFitDataFrame(pred_data,QUESTION)
#     val_xx = pred_data[col_fit2]
#     val_yy = pred_data["answered_correctly"]
#     pred = model.predict_proba(val_xx)
#     preds = [*preds,*pred[:,1]]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as pl
pl.figure(figsize=(15,8))
sns.barplot(y=train_x.columns, x=model.feature_importances_, )

In [ ]:
del train,train_x,train_y,val_x,val_y,DATA_TRAIN,DATA_VAL
gc.collect()

In [ ]:
# start_time= time.time()
# example_test = pd.read_csv("../input/riiid-test-answer-prediction/example_test.csv")
# example_test = constructFitDataFrame(example_test,QUESTION,"TEST")
# example_test['answered_correctly'] =  model.predict_proba(example_test[col_fit2])[:,1]
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
start_time= time.time()
CONTEXT_STAT3 = 0
COLUMNS_SAVE = ["content_id","correct","user_id","content_type_id","prior_question_had_explanation"]
for (test_df, sample_prediction_df) in iter_test:
    test_df = constructFitDataFrame(test_df,QUESTION,"TEST")
    test_df['answered_correctly'] =  model.predict_proba(test_df[col_fit2])[:,1]
#     display(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
print("--- %s seconds ---" % (time.time() - start_time))